The following cell takes our super covid dataset from stage I and filters out dates not in the range of June 01, 2020 to January 3, 2021. It also filters out US states so that I only have the 6 US states that I want. In my case, I'm analyzing New York, Texas, Illinois, Arizona, Colorado, and North Dakota.

In [35]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

super_covid19_data = pd.read_csv('super_covid19_data.csv')

#Filter for cases between June 01, 2020 and January 3 2021
start_date = '2020-06-01'
end_date =  '2021-01-04' 


# Filter for columns between the start_date and end_date for cases and deaths
all_columns = super_covid19_data.columns
filtered_columns = all_columns[(all_columns >= start_date) & (all_columns <= end_date)]

# Create a filtered DataFrame with only the relevant date columns
filtered_super_covid19_data = super_covid19_data[['countyFIPS', 'State', 'County Name_x', 'population'] + list(filtered_columns)]

#US state abbreviations: NY, TX, IL, AZ, CO, ND
states_to_keep = ['NY', 'TX', 'IL', 'AZ', 'CO', 'ND']

# Filter the DataFrame to keep only the rows for these states
filtered_super_covid19_data = filtered_super_covid19_data[filtered_super_covid19_data['State'].isin(states_to_keep)]

# Display the filtered dataset
print(filtered_super_covid19_data.head())

# Save the filtered dataset to a CSV file
filtered_super_covid19_data.to_csv('filtered_super_dates_June2020_Jan2021_and_6_states.csv', index=False)

     countyFIPS State          County Name_x  population  2020-06-01_cases  \
98            0    AZ  Statewide Unallocated           0                 0   
99         4001    AZ         Apache County        71887              1526   
100        4003    AZ        Cochise County       125922                76   
101        4005    AZ       Coconino County       143476              1155   
102        4007    AZ           Gila County        54018                31   

     2020-06-02_cases  2020-06-03_cases  2020-06-04_cases  2020-06-05_cases  \
98                  0                 0                 0                 0   
99               1569              1586              1637              1656   
100                89                94               105               120   
101              1173              1186              1221              1248   
102                35                37                39                42   

     2020-06-06_cases  ...  2020-12-25_deaths  2020-12-2